In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterCRUD import AnimalShelterCRUD





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelterCRUD(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))






#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/liamnunes_snhu/Documents/Final Project/assets/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.H1("Liam Nunes"),
    dcc.RadioItems(
        id = 'types',
        options = [
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
    ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,
        sort_action='native',
        sort_mode='multi'
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
             [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    #adjusts the read request for the desired dog type and status
    if filter_type == 'Water Rescue':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                      {'$or': [
                                                          {'breed': 'Labrador Retriever Mix'},
                                                          {'breed': 'Chesa Bay Retr Mix'},
                                                          {'breed': 'Newfoundland'},
                                                          },
                                                      {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                          {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                               }]})))
    elif filter_type == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                         {'$or': [
                                                             {'breed': 'German Shepherd'},
                                                             {'breed': 'Alaskan Malamute'},
                                                             {'breed': 'Old English Sheepdog'},
                                                             {'breed': 'Rottweiler'},
                                                             {'breed': 'Siberian Husky'}]
                                                         },
                                                         {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                   {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                         }]
                                               })))
    elif filter_type == 'Disaster Rescue or Individual Tracking':       
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                         {'$or': [
                                                             {'breed': 'Doberman Pinscher'},
                                                             {'breed': 'German Shepherd'},
                                                             {'breed': 'Golden Retriever'},
                                                             {'breed': 'Bloodhound'},
                                                             {'breed': 'Rottweiler'}]
                                                         },
                                                         {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                   {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                         }]
                                               })))
    elif filter_type == 'Reset':
        df = pd.DataFrame.from_records(shelter.read({}))

       
    clumns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values=values, names=names, title='Breeds availible')
        )    
    ]

@app.callback(
    Output('map-id', "children"), 
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app